In [1]:
import pandas as pd
import numpy as np

#### Dataset Descriptions
- **orders**: Contains one row per order, including user ID, order sequence number,
  order timing information, and evaluation split (prior / train / test).
- **order_products__prior**: Historical product-level purchase records used to
  construct user and product behavioral features.
- **order_products__train**: Training labels indicating whether a product was reordered.
- **products**: Product metadata including aisle and department identifiers.
- **aisles**: Maps aisle IDs to aisle names.
- **departments**: Maps department IDs to department names.

In [21]:
orders = pd.read_csv('/Users/mr.tian/Desktop/archive/orders.csv')
opp = pd.read_csv('/Users/mr.tian/Desktop/archive/order_products__prior.csv')
opt = pd.read_csv('/Users/mr.tian/Desktop/archive/order_products__train.csv')
products = pd.read_csv('/Users/mr.tian/Desktop/archive/products.csv')
aisles = pd.read_csv('/Users/mr.tian/Desktop/archive/aisles.csv')
departments = pd.read_csv('/Users/mr.tian/Desktop/archive/departments.csv')

#### Build Training Labels
Merge the training order-product table with order-level information in order to
associate each product with a specific user. The target variable is `reordered`.

In [22]:
opt_new = opt.merge(orders, how='left', on='order_id')[['user_id', 'product_id', 'reordered']]
opt_new.head()

,user_id,product_id,reordered
0,112108,49302,1
1,112108,11109,1
2,112108,10246,0
3,112108,49683,0
4,112108,43633,1


In [23]:
orders['eval_set'].unique()

array(['prior', 'train', 'test'], dtype=object)

#### Filter Prior Orders for Feature Engineering
Only prior orders are used to construct historical user-level features in order
to avoid data leakage from training or test orders.

In [24]:
orders = orders[orders['eval_set'] == 'prior']

#### User-Level Feature: Total Number of Orders
For each user, the maximum order number corresponds to the total number of orders placed.

In [25]:
orders_new = orders.groupby('user_id', as_index=False)['order_number'].max()

#### User-Level Feature: Average Days Between Orders
Compute the average number of days between consecutive orders for each user,
which captures purchasing frequency.

In [44]:
user_gaps = orders.groupby('user_id')['days_since_prior_order'].mean().rename({'days_since_prior_order':'mean_days_since_last_order'})

#### Merge User-Level Features
Combine all user-level features into a single table for downstream modeling.

In [46]:
orders_new = orders_new.merge(user_gaps, how='left', on='user_id')

#### User-Level Feature: Average Order Hour
Compute the average hour of day when each user places orders, which reflects
their typical shopping time preference.

In [49]:
user_order_hod = orders.groupby('user_id')['order_hour_of_day'].mean()

#### User-Level Feature: Most Frequent Order Day of Week
Identify the most common day of week on which each user places orders.
This feature captures weekly purchasing patterns.


In [51]:
user_dow = orders.groupby('user_id').agg(user_dow=('order_dow', lambda s: s.mode().iloc[0]))

#### Merge User Temporal Features
Merge the average order hour and most frequent order day into the
user-level feature table.


In [55]:
orders_new = orders_new.merge(user_order_hod, how='left', on='user_id')
orders_new = orders_new.merge(user_dow, how='left', on='user_id')

In [56]:
orders_new.head()

,user_id,order_number,days_since_prior_order,order_hour_of_day,user_dow
0,1,10,19.555556,10.300000,1
1,2,14,15.230769,10.571429,1
2,3,12,12.090909,16.416667,0
3,4,5,13.750000,12.600000,4
4,5,4,13.333333,16.000000,3


#### Product-Level Features: Purchase Frequency and Reorder Rate
Compute how frequently each product appears in prior orders and
the proportion of times it was reordered.

In [61]:
product_feat = (
    opp.groupby('product_id')
        .agg(
            product_cnt=('order_id', 'count'), 
            product_reorder_rate=('reordered', 'mean')
        )
)

#### Attach User Information to Prior Order-Product Data
Merge user IDs into the prior order-product table to enable
product-level user statistics.


In [63]:
opp_user = opp.merge(orders[['order_id', 'user_id']], on='order_id', how='left')

#### Product-Level Feature: Number of Unique Users
Calculate how many distinct users have purchased each product,
which reflects product popularity across the user base.


In [65]:
product_user = (
    opp_user.groupby('product_id', as_index=False)
        .agg(product_unique_user=('user_id', 'nunique'))
)

#### Combine Product-Level Features
Merge the unique-user count into the existing product feature table so that each product has:
(1) total purchase count, (2) reorder rate, and (3) number of unique buyers.


In [67]:
product_feat = product_feat.merge(product_user, how='left', on='product_id')

In [68]:
product_feat.head()

,product_id,product_cnt,product_reorder_rate,product_unique_user
0,1,1852,0.613391,716
1,2,90,0.133333,78
2,3,277,0.732852,74
3,4,329,0.446809,182
4,5,15,0.600000,6


#### Sanity Checks for Product Features
1) Ensure each product_id appears only once in the product feature table.
2) Ensure product_reorder_rate is a valid probability between 0 and 1.


In [74]:
print(product_feat['product_id'].is_unique)
print(product_feat['product_reorder_rate'].between(0, 1).all())

True
True


#### Build User–Product Interaction Base Table (Prior Data)
Attach user_id and order_number to the prior order-product table.
This enables user–product level features such as:
- how many times a user purchased a product
- when the user last purchased the product (latest order_number)


In [76]:
opp_u = opp.merge(orders[['order_id', 'user_id', 'order_number']], on='order_id', how='left')

#### User–Product Feature: Purchase Count
For each (user_id, product_id) pair, count how many times the user purchased that product in prior orders.


In [79]:
up_cnt = (
    opp_u.groupby(['user_id', 'product_id'], as_index=False)
        .agg(up_cnt=('order_id', 'count'))
)

#### User–Product Feature: Most Recent Purchase Order
For each (user_id, product_id) pair, get the maximum order_number,
which indicates the user's most recent order that included the product.


In [81]:
up_last_order = (
    opp_u.groupby(['user_id', 'product_id'], as_index=False)
         .agg(up_last_order=('order_number', 'max'))
)

#### User–Product Feature: Purchase Ratio (User-Level Normalization)
Merge user total order count into the user–product table and compute:

ratio = (number of times user bought the product) / (total number of user orders)

This measures how frequently the user buys the product relative to their overall ordering behavior.


In [83]:
up_feat = up_cnt.merge(orders_new[['user_id', 'order_number']], on='user_id', how='left')

In [84]:
up_feat['ratio'] = up_feat['up_cnt'] / up_feat['order_number']

#### Merge All Features into the Training Table
Join user-level features, product-level features, and user–product interaction features
into the labeled training dataset `opt_new`.


In [92]:
opt_new = opt_new.merge(orders_new, on='user_id', how='left')
opt_new = opt_new.merge(product_feat, on='product_id', how='left')
opt_new = opt_new.merge(up_feat, on=['user_id', 'product_id'], how='left')

#### Rename Columns for Clarity
Rename merged columns to make feature names clearer and avoid confusion caused by suffixes.


In [94]:
opt_new = opt_new.rename(
    columns={
        'order_number_x': 'user_order_cnt',
        'order_number_y': 'up_last_order'
    }
)

In [98]:
opt_new = opt_new.rename(columns={'ratio': 'up_ratio'})

#### Handle Missing Values for User–Product Interaction Features
Fill missing values with zero. A missing value indicates that the user has never
purchased the product before.


In [100]:
opt_new['up_cnt'] = opt_new['up_cnt'].fillna(0)
opt_new['up_last_order'] = opt_new['up_last_order'].fillna(0)
opt_new['up_ratio'] = opt_new['up_ratio'].fillna(0)

#### Handle Missing Values for Product-Level Features
Fill missing values for product statistics with zero, which typically corresponds
to products with very limited historical data.


In [109]:
cols = ['product_cnt', 'product_reorder_rate', 'product_unique_user']
opt_new[cols] = opt_new[cols].fillna(0)

In [110]:
opt_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384617 entries, 0 to 1384616
Data columns (total 13 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   user_id                 1384617 non-null  int64  
 1   product_id              1384617 non-null  int64  
 2   reordered               1384617 non-null  int64  
 3   user_order_cnt          1384617 non-null  int64  
 4   days_since_prior_order  1384617 non-null  float64
 5   order_hour_of_day       1384617 non-null  float64
 6   user_dow                1384617 non-null  int64  
 7   product_cnt             1384617 non-null  float64
 8   product_reorder_rate    1384617 non-null  float64
 9   product_unique_user     1384617 non-null  float64
 10  up_cnt                  1384617 non-null  float64
 11  up_last_order           1384617 non-null  float64
 12  up_ratio                1384617 non-null  float64
dtypes: float64(8), int64(5)
memory usage: 137.3 MB


#### Organize Features by Category
Group features into user-level, product-level, and user–product interaction features.
This modular design makes feature selection and ablation easier in later experiments.


In [124]:
USER_FEATURE = [
    'user_order_cnt',
    'days_since_prior_order',
    'order_hour_of_day',
    'user_dow'
]
PRODUCT_FEATURE = [
    'product_cnt',
    'product_unique_user'
]
UP_FEATURE = [
    'up_cnt',
    'up_last_order',
    'up_ratio'
]
FEATURE = USER_FEATURE + PRODUCT_FEATURE

#### Define Features and Target Variable
Split the dataset into input features (X) and target variable (y).
The target variable indicates whether a product was reordered.


In [125]:
X = opt_new[FEATURE]
y = opt_new['reordered']

#### Train–Test Split
Split the dataset into training and testing sets using an 80/20 ratio.


In [126]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Log Transformation for Long-Tailed Features
Apply log1p transformation to highly skewed count-based features to
stabilize model training and reduce the impact of extreme values.

In [139]:
X_train2 = X_train.copy()
X_test2 = X_test.copy()

for c in ["product_cnt", "product_unique_user", "user_order_cnt"]:
    X_train2[c] = np.log1p(X_train2[c])
    X_test2[c]  = np.log1p(X_test2[c])

#### Model Definitions
Define multiple baseline models for comparison, including:
- Logistic Regression (linear baseline with regularization)
- K-Nearest Neighbors (distance-based model)
- Random Forest (tree-based ensemble model)

All models are trained and evaluated on the same feature set to ensure a fair comparison.


#### Train and Evaluate Baseline Models
Train each model on the training set and evaluate performance on the test set
using accuracy, precision, recall, and F1-score.

This step helps identify the most suitable model architecture for the task.


In [147]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer


models = {
    "Logistic(saga)": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(
            solver="saga", penalty="l2", C=0.1,
            max_iter=5000, n_jobs=1, class_weight="balanced"
        ))
    ]),
    "KNN": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", KNeighborsClassifier())
    ]),
    "RandomForest": RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42, max_depth=12, min_samples_leaf=50)
}

for name, model in models.items():
    model.fit(X_train2, y_train)     
    pred = model.predict(X_test2)
    print(name,
          f"acc={accuracy_score(y_test,pred):.4f}",
          f"prec={precision_score(y_test,pred):.4f}",
          f"rec={recall_score(y_test,pred):.4f}",
          f"f1={f1_score(y_test,pred):.4f}")

Logistic(saga) acc=0.6600 prec=0.7515 rec=0.6447 f1=0.6940
KNN acc=0.6408 prec=0.6857 rec=0.7377 f1=0.7107
RandomForest acc=0.6732 prec=0.6959 rec=0.8059 f1=0.7469


#### Threshold Tuning for Random Forest
Instead of using the default classification threshold (0.5), evaluate multiple thresholds
based on predicted probabilities to analyze the precision–recall trade-off.

The goal is to select a threshold that maximizes the F1-score while maintaining high recall.


In [148]:
proba = models["RandomForest"].predict_proba(X_test2)[:, 1]

for t in [0.3, 0.4, 0.5, 0.6]:
    pred_t = (proba >= t).astype(int)
    print("thr", t,
          f"prec={precision_score(y_test,pred_t):.4f}",
          f"rec={recall_score(y_test,pred_t):.4f}",
          f"f1={f1_score(y_test,pred_t):.4f}")

thr 0.3 prec=0.6059 rec=0.9944 f1=0.7530
thr 0.4 prec=0.6399 rec=0.9432 f1=0.7625
thr 0.5 prec=0.6959 rec=0.8059 f1=0.7469
thr 0.6 prec=0.7577 rec=0.6071 f1=0.6741


#### Threshold Comparison Results
Lower thresholds increase recall at the cost of precision, while higher thresholds
produce more conservative predictions.

A threshold of 0.4 provides the best balance, achieving the highest F1-score.


In [153]:
final_threshold = 0.4

proba = models["RandomForest"].predict_proba(X_test2)[:, 1]
final_pred = (proba >= final_threshold).astype(int)

#### Final Model Selection
Based on the evaluation results, select:
- Model: Random Forest
- Decision threshold: 0.4

This configuration offers the best overall balance between precision and recall.


In [154]:
final_model = models["RandomForest"]
final_threshold = 0.4

#### Save the Final Model
Persist the trained Random Forest model to disk using joblib.
This allows the model to be reused without retraining.


In [155]:
from joblib import dump
dump(final_model, "rf_final_model.joblib")

['rf_final_model.joblib']

#### Final Model Evaluation
Generate a detailed classification report for the final model,
including class-wise precision, recall, and F1-score.

This report provides a comprehensive view of model performance,
especially for the positive (reordered) class.


In [156]:
from sklearn.metrics import classification_report
print(classification_report(y_test, final_pred))

              precision    recall  f1-score   support

           0       0.71      0.21      0.32    111266
           1       0.64      0.94      0.76    165658

    accuracy                           0.65    276924
   macro avg       0.68      0.58      0.54    276924
weighted avg       0.67      0.65      0.59    276924

